### Decorators

Good resource for the basics: https://realpython.com/primer-on-python-decorators/

The purpose of these: Wrap a function, modifying the behavior of said function. Basic example below:

In [1]:
# without decorators:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

# decoration 
say_whee = my_decorator(say_whee)

# say_whee -> this is just going to be a reference to "wrapper"
print(say_whee) # local var in my_decorator

<function my_decorator.<locals>.wrapper at 0x1029c8430>


In [2]:
# Maybe a more relevant example without decorator
# We can add logic to limit when our function will actually be evaluated
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

# decoration changes behavior of say_whee, reducing time it can run
say_whee = not_during_the_night(say_whee)

print(f"The hour is: {datetime.now().hour}")
print(f"rep: {say_whee}")

say_whee()

The hour is: 16
rep: <function not_during_the_night.<locals>.wrapper at 0x1029c8430>
Whee!


### Syntactic Sugar: The above example is clunky

Why? We have to call `say_whee` 3 separate times. Instead we can lean on the `@` syntax and solve the same problem as above. 

In [3]:
@not_during_the_night
def say_whee():
    print("Whee!")
    
print(f"The hour is: {datetime.now().hour}")
print(f"rep: {say_whee}")

say_whee()

The hour is: 16
rep: <function not_during_the_night.<locals>.wrapper at 0x102a01040>
Whee!


### Some More Examples:

#### Run a function twice

In [4]:
def do_twice(func):
    """Decorate that will run inner function twice"""
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")
    
# Now we will see our function run twice:
print(f"rep: {say_whee}")
say_whee()

rep: <function do_twice.<locals>.wrapper_do_twice at 0x102a01280>
Whee!
Whee!


#### Decorate with Arguments; 

We need to make a minor shift since our inner function does not currently accept any arguments. 

We need to use `*args` and `**kwargs`. The function`do_twice` below will not accept any number of arguments and pass them to the function it decorates. 

In [5]:
def do_twice(func):
    """Decorate that will run inner function twice with optional arguments"""
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")
    
@do_twice
def greet(name):
    print(f"Hello {name}")
    
@do_twice
def greet_age(name, weeks):
    print(f"Hello {name}, you are {weeks} weeks old today!")
    
# Now we will see our function run twice:
say_whee()

# Now we will see our function run twice with arguments
greet("Thomas")

# Any more arguments!
greet_age("Thomas", 5)

Whee!
Whee!
Hello Thomas
Hello Thomas
Hello Thomas, you are 5 weeks old today!
Hello Thomas, you are 5 weeks old today!


### Returning Values from A Decorated Function

We can't just return a value, see below

In [6]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_adam = return_greeting("Adam")


print(hi_adam)


Creating greeting
Creating greeting
None


What is happening? 

The decorator is "eating" the return value from the function. `wrapper_do_twice` doesn't explicitly return a value, so we need to do add a return in the wrapper function.

In [15]:
def do_twice(func):
    """Test Doc 1"""
    def wrapper_do_twice(*args, **kwargs):
        """Test doc 2"""
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def return_greeting(name):
    """Return greeting"""
    print("Creating greeting")
    return f"Hi {name}"

hi_adam = return_greeting("Adam")


print(hi_adam)


Creating greeting
Creating greeting
Hi Adam


### Adding a Clearer Name

Right now the introspection shows a confusing output

In [17]:
print(return_greeting) # says it is the "wrapper_do_twice" function inside of "function_do_twice"
print(return_greeting.__name__)
print(return_greeting.__doc__) # returns the doc for the wrong function!

<function do_twice.<locals>.wrapper_do_twice at 0x1077a3e50>
wrapper_do_twice
Test doc 2


In [18]:
# let's fix this 
import functools

# we can use the functools.wraps decorate, which will preserve information about the original function
def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def return_greeting(name):
    """Return greeting"""
    print("Creating greeting")
    return f"Hi {name}"

print(return_greeting) # properly captures "return_greeting"
print(return_greeting.__name__)
print(return_greeting.__doc__) # returns the doc for "return greeting"

<function return_greeting at 0x1077a3d30>
return_greeting
Return greeting


### Cool Real World Examples

#### Timing functions

This is pretty nice - we can measure the time a function takes to execute instead of just adding start / end code


In [19]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1: Start time prior to function run
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2: End time after function run
        run_time = end_time - start_time    # 3: Determine diff
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs") # 4: Print total time for specific function to run
        return value # 5: Make sure we return the value 
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [20]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0049 secs


In [21]:
waste_some_time(999)

Finished 'waste_some_time' in 2.0878 secs


#### Debugger

In [28]:
def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1: create a list of arguments (we use repr to get string rep)
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2: generate list of keyword args
        signature = ", ".join(args_repr + kwargs_repr)           # 3: Generate a single list of args & kwargs
        print(f"Calling {func.__name__}({signature})")           # 4: Print the func name and list of args/kwargs
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 5: print return value 
        return value # 6: the real return
    return wrapper_debug

In [29]:
import math

# Apply a decorator to track recursive func of math.factorial
@debug
def get_factorial_recursively(n):
    if n <= 1:
        return 1
    else:
        return n * get_factorial_recursively(n-1)

In [30]:
output = get_factorial_recursively(5)
assert(output == math.factorial(5))

Calling get_factorial_recursively(5)
Calling get_factorial_recursively(4)
Calling get_factorial_recursively(3)
Calling get_factorial_recursively(2)
Calling get_factorial_recursively(1)
'get_factorial_recursively' returned 1
'get_factorial_recursively' returned 2
'get_factorial_recursively' returned 6
'get_factorial_recursively' returned 24
'get_factorial_recursively' returned 120
Calling factorial(5)
'factorial' returned 120
